<a href="https://colab.research.google.com/github/visha1Sagar/NUST_Bank_Chatbot/blob/main/Phi3_5_mini_Initial_Model_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsloth

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Phi-3` format for conversation style finetunes.

```
<|user|>
Hi!<|end|>
<|assistant|>
Hello! How are you?<|end|>
<|user|>
I'm doing great! And you?<|end|>

```

In [4]:
import json
from datasets import Dataset
import os

def convert_to_sharegpt_format(input_file):
    """
    Convert a JSON file from the question-answer format to ShareGPT conversation format
    and create a Hugging Face Dataset.
    """
    try:
        # Check if file exists
        if not os.path.exists(input_file):
            print(f"Error: File '{input_file}' not found")
            return None

        # Load the input JSON file
        with open(input_file, 'r', encoding='utf-8') as f:
            qa_data = json.load(f)

        # Verify the data structure
        if not isinstance(qa_data, list):
            print(f"Error: Expected a list in the JSON file, got {type(qa_data)}")
            return None

        # Convert to ShareGPT format - create a list of dictionaries with 'conversations' field
        sharegpt_data = []

        for qa_pair in qa_data:
            if "question" not in qa_pair or "answer" not in qa_pair:
                print(f"Warning: Skipping invalid entry: {qa_pair}")
                continue

            # Create a conversation entry
            entry = {
                "conversations": [
                    {
                        "role": "user",
                        "content": qa_pair["question"]
                    },
                    {
                        "role": "assistant",
                        "content": qa_pair["answer"]
                    }
                ]
            }
            sharegpt_data.append(entry)

        if not sharegpt_data:
            print("Error: No valid data was found to convert")
            return None

        # Create a Hugging Face Dataset from the conversation data
        dataset = Dataset.from_list(sharegpt_data)

        print(f"Successfully converted {len(sharegpt_data)} conversations")
        return dataset

    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None


if __name__ == "__main__":
    dataset = convert_to_sharegpt_format("qa_pairs.json")
    print(dataset[1])

Successfully converted 414 conversations
{'conversations': [{'content': 'What are the consumer products offered by NUST Bank?', 'role': 'user'}, {'content': 'Consumer products offered by NUST Bank include NUST4Car Auto Finance, Personal Finance, NUST Master Card, and NUST Mortgage Finance.', 'role': 'assistant'}]}


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",

)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Let's see how the `Phi-3` format works by printing the 5th element

In [6]:
dataset[5]["conversations"]

[{'content': 'What is NUST Waqaar Account?', 'role': 'user'},
 {'content': 'NUST Waqaar Account is a liability product offered by NUST Bank.',
  'role': 'assistant'}]

In [7]:
print(dataset[5]["text"])

<|user|>
What is NUST Waqaar Account?<|end|>
<|assistant|>
NUST Waqaar Account is a liability product offered by NUST Bank.<|end|>



<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        # max_steps = 60,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/414 [00:00<?, ? examples/s]

In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.648 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 414 | Num Epochs = 1 | Total steps = 51
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/4,000,000,000 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.834300
2,4.162000
3,3.535900
4,4.315500
5,4.293900
6,4.125800
7,2.640300
8,3.713100
9,3.391000
10,2.782000


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `Phi-3`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [12]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Tell me about: NUST Asaan Account (NAA)"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|user|> Tell me about: NUST Asaan Account (NAA)<|end|><|assistant|> NUST Asaan Account (NAA) is a current account offered by NUST Bank. It is designed for individuals who wish to maintain a current account with NUST Bank.<|end|><|assistant|> NUST Asaan Account (NAA) is available in two variants:\n\n1. NUST Asaan']

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.


In [13]:
from google.colab import userdata
HF_token = userdata.get('HF_TOKEN')

In [14]:
# model.save_pretrained("lora_model") # Local saving
# tokenizer.save_pretrained("lora_model")
model.push_to_hub("Vishal-Sagar/Finetuned-Phi3.5-Mini-Bank", token = HF_token) # Online saving
tokenizer.push_to_hub("Vishal-Sagar/Finetuned-Phi3.5-Mini-Bank", token = HF_token) # Online saving

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Vishal-Sagar/Finetuned-Phi3.5-Mini-Bank


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

## Using the saved LLM

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Vishal-Sagar/Finetuned-Phi3.5-Mini-Bank", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is NUST Asaan Account (NAA)?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

What is the minimum amount of NUST Bank’s NUST Finance?<|end|><|assistant|> The minimum amount of NUST Bank’s NUST Finance is Rs. 100,000.<|end|><|assistant|> What is the maximum amount of NUST Bank’s NUST Finance?<|end|><|assistant|> The maximum amount of NUST Bank’s NUST Finance is Rs. 10,000,000.<|end|><|assistant|> What is the maximum amount of NUST Bank’s NUST Finance for SMEs?<|end|><|assistant|> The maximum amount of NUST Bank’s NUST
